In [14]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [2]:
from pyspark import SparkContext

sc = SparkContext()

In [3]:
sample_2016 = sc.textFile("s3://gdelt-open-data/events/2016*")

In [6]:
parts = sample_2016.map(lambda l: l.split('\t'))


In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("practice").getOrCreate()

In [30]:
from urllib.request import urlopen
html = urlopen("http://gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt").read().rstrip().decode('utf-8')
columns = html.split('\t')
df = spark.createDataFrame(parts, columns)

In [35]:
df.createOrReplaceTempView("gdelt")

In [42]:
sqlWay = spark.sql("""
SELECT Actor1Code, Actor1Name, Actor1CountryCode, Actor1Religion1Code
FROM gdelt
LIMIT 5
""")
sqlWay.explain()

== Physical Plan ==
CollectLimit 5
+- *(1) Project [Actor1Code#645, Actor1Name#646, Actor1CountryCode#647, Actor1Religion1Code#650]
   +- Scan ExistingRDD[GLOBALEVENTID#640,SQLDATE#641,MonthYear#642,Year#643,FractionDate#644,Actor1Code#645,Actor1Name#646,Actor1CountryCode#647,Actor1KnownGroupCode#648,Actor1EthnicCode#649,Actor1Religion1Code#650,Actor1Religion2Code#651,Actor1Type1Code#652,Actor1Type2Code#653,Actor1Type3Code#654,Actor2Code#655,Actor2Name#656,Actor2CountryCode#657,Actor2KnownGroupCode#658,Actor2EthnicCode#659,Actor2Religion1Code#660,Actor2Religion2Code#661,Actor2Type1Code#662,Actor2Type2Code#663,... 34 more fields]


In [60]:
#output here looks nice, but not sure if table set up correctly (i.e. Actor1Name being a column and the rows
#seem like other column records)
from pyspark.sql.functions import window, column, desc, col

df\
.selectExpr("Actor1Code", 
              "Actor1Name", 
              "Actor1CountryCode", 
              "Actor1Religion1Code",
           "NumMentions")\
.groupBy(col("Actor1Name"))\
.count()\
.show(5)


+----------+-----+
|Actor1Name|count|
+----------+-----+
|    SOMALI|32047|
|   ARMENIA|80388|
|      A US|28942|
|     OSAGE| 1632|
|    HUNTER|32782|
+----------+-----+
only showing top 5 rows



In [62]:
df\
.selectExpr("GLOBALEVENTID",
            "EventCode",
            "EventBaseCode",
            "EventRootCode",
            "NumMentions", 
            "NumSources",
            "NumArticles")\
.show(5)

+-------------+---------+-------------+-------------+-----------+----------+-----------+
|GLOBALEVENTID|EventCode|EventBaseCode|EventRootCode|NumMentions|NumSources|NumArticles|
+-------------+---------+-------------+-------------+-----------+----------+-----------+
|    498554164|      051|          051|           05|          6|         2|          6|
|    498554165|      051|          051|           05|          6|         2|          6|
|    498554166|      070|          070|           07|         12|         2|         12|
|    498554167|      051|          051|           05|          6|         2|          6|
|    498554168|      051|          051|           05|          2|         2|          2|
+-------------+---------+-------------+-------------+-----------+----------+-----------+
only showing top 5 rows

